 # Imports and Setup

In [7]:
import os
from pathlib import Path
import pyvista as pv
import src  # Assuming 'src' contains the hiFEM function

# Ensure the output directory exists
os.makedirs("outputs", exist_ok=True)


 # Case A

 ## Load common inputs

In [8]:
# Load the main resection surface
resection_mesh_CaseA = pv.read(Path("inputs/CaseA_Resection.obj")).triangulate()
resection_verts_CaseA = resection_mesh_CaseA.points
resection_faces_CaseA = resection_mesh_CaseA.faces.reshape(-1, 4)[:, 1:]

# Load the scene object (for illustration purposes only)
scene_obj_CaseA = pv.read(Path("inputs/CaseA_TongueResected.obj"))


 ## hiFEM

In [9]:
pmesh_CaseA_hiFEM, flap_mesh_CaseA_hiFEM = src.hiFEM(
    verts3D=resection_verts_CaseA,
    faces=resection_faces_CaseA,
    thickness=2.0,
    nu=0.49,
    max_iters=30,
    convergence_tol=1e-10,
    verbose=True,
    scene_obj=scene_obj_CaseA, # Use pre-loaded scene object
)
# Output file: result_flap_CaseA_hiFEM.obj
pmesh_CaseA_hiFEM.save("outputs/result_flap_CaseA_hiFEM.obj", binary=False)


Step 0: L2norm = 72.07071576953102
Step 1: L2norm = 0.0464763592476186
Step 2: L2norm = 0.002797927334789039
Step 3: L2norm = 0.00018965492551395048
Step 4: L2norm = 1.264006950359719e-05
Step 5: L2norm = 8.272118450801541e-07
Step 6: L2norm = 5.3134874055429676e-08
Step 7: L2norm = 3.349751801939317e-09
Step 8: L2norm = 2.0789372807819824e-10
Step 9: L2norm = 1.3989644139259657e-11


 ## iFEM
 When Hollman (power law constitutive model) is used, we are effectively using iFEM.

In [10]:
pmesh_CaseA_iFEM, flap_mesh_CaseA_iFEM = src.hiFEM(
    verts3D=resection_verts_CaseA,
    faces=resection_faces_CaseA,
    thickness=2.0,
    nu=0.49,
    material_model="Hollman",
    max_iters=30,
    convergence_tol=1e-10,
    verbose=True,
    scene_obj=scene_obj_CaseA, # Use pre-loaded scene object
)
# Output file: result_flap_CaseA_iFEM.obj
pmesh_CaseA_iFEM.save("outputs/result_flap_CaseA_iFEM.obj", binary=False)


Step 0: L2norm = 71.95488341480534
Step 1: L2norm = 0.15739210500355305
Step 2: L2norm = 0.015507324521815299
Step 3: L2norm = 0.003034555585519371
Step 4: L2norm = 0.0008034323816038404
Step 5: L2norm = 0.0002479479951563962
Step 6: L2norm = 8.340353124073157e-05
Step 7: L2norm = 2.9603728230692716e-05
Step 8: L2norm = 1.0860617288916496e-05
Step 9: L2norm = 4.067875613775204e-06
Step 10: L2norm = 1.543942789162261e-06
Step 11: L2norm = 5.911447335373675e-07
Step 12: L2norm = 2.276886029828004e-07
Step 13: L2norm = 8.806429103072456e-08
Step 14: L2norm = 3.4163155022975384e-08
Step 15: L2norm = 1.3282151647054068e-08
Step 16: L2norm = 5.1723325100216315e-09
Step 17: L2norm = 2.0166779043580993e-09
Step 18: L2norm = 7.870520269657049e-10
Step 19: L2norm = 3.07382526239556e-10
Step 20: L2norm = 1.201150881604251e-10
Step 21: L2norm = 4.696039272256747e-11


 ## BFF
The hiFEM function initially generates a guess using the BFF method. When the parameter `fix_all_dofs` is set to `True`, the function does not update this initial guess. Therefore, the resulting flap design is identical to the one initially generated by BFF.

P.S. The function calculates the deformation metrics as before.


In [11]:
pmesh_CaseA_BFF, flap_mesh_CaseA_BFF = src.hiFEM(
    verts3D=resection_verts_CaseA,
    faces=resection_faces_CaseA,
    thickness=2.0,
    nu=0.49,
    fix_all_dofs=True,
    max_iters=30,               # Iterations not performed, but parameter needed
    convergence_tol=1e-10,      # Convergence not performed, but parameter needed
    verbose=True,
    scene_obj=scene_obj_CaseA, # Use pre-loaded scene object
)
# Output file: result_flap_CaseA_BFF.obj
pmesh_CaseA_BFF.save("outputs/result_flap_CaseA_BFF.obj", binary=False)

Step 0: L2norm = 72.03864591884836
Step 1: L2norm = 0.0


 ## NURBS flattening (Metrics Calculation)
Since the NURBS flattening generated flap is provided, we set `fix_all_dofs=True`. The function will use the provided NURBS flap and the associated resection surface solely to calculate deformation metrics without optimizing the flap shape further.


In [12]:
# Load the specific resection surface associated with the NURBS generation process.
resection_mesh_NURBS_CaseA = pv.read(Path("inputs/CaseA_NURBS_resection.obj")).triangulate()
resection_verts_NURBS_CaseA = resection_mesh_NURBS_CaseA.points
resection_faces_NURBS_CaseA = resection_mesh_NURBS_CaseA.faces.reshape(-1, 4)[:, 1:]

# Load the NURBS-generated flattened flap vertices (initial guess).
ig_verts_CaseA = pv.read(Path("inputs/CaseA_NURBS_flap.obj")).triangulate().points[:, :2]

pmesh_CaseA_NURBS, flap_mesh_CaseA_NURBS = src.hiFEM(
    verts3D=resection_verts_NURBS_CaseA, # Use NURBS-specific resection vertices
    faces=resection_faces_NURBS_CaseA,   # Use NURBS-specific resection faces
    thickness=2.0,
    nu=0.49,
    ig_verts=ig_verts_CaseA,          # Use NURBS-generated flap as initial guess
    fix_all_dofs=True,                # Fix DoFs to only calculate metrics
    max_iters=30,                     # Iterations not performed, but parameter needed
    convergence_tol=1e-10,            # Convergence not checked, but parameter needed
    verbose=True,
    scene_obj=scene_obj_CaseA,        # Use the standard scene object for context
)
# Save the NURBS flap mesh with calculated metrics
# Output file: result_flap_CaseA_NURBS.obj
pmesh_CaseA_NURBS.save("outputs/result_flap_CaseA_NURBS.obj", binary=False)



Step 0: L2norm = 103.81846119328915
Step 1: L2norm = 0.0


 # Case B
(Repitition in the comments is avoided. It follows the same pattern as in Case A.)

 ## Load common inputs

In [13]:
# Load the main resection surface
resection_mesh_CaseB = pv.read(Path("inputs/CaseB_Resection.obj")).triangulate()
resection_verts_CaseB = resection_mesh_CaseB.points
resection_faces_CaseB = resection_mesh_CaseB.faces.reshape(-1, 4)[:, 1:]

# Load the scene object
scene_obj_CaseB = pv.read(Path("inputs/CaseB_TongueResected.obj"))


 ## hiFEM

In [14]:
pmesh_CaseB_hiFEM, flap_mesh_CaseB_hiFEM = src.hiFEM(
    verts3D=resection_verts_CaseB,
    faces=resection_faces_CaseB,
    thickness=2.0,
    nu=0.49,
    max_iters=30,
    convergence_tol=1e-10,
    verbose=True,
    scene_obj=scene_obj_CaseB,
)
# Output file: result_flap_CaseB_hiFEM.obj
pmesh_CaseB_hiFEM.save("outputs/result_flap_CaseB_hiFEM.obj", binary=False)


Step 0: L2norm = 87.46965094243973
Step 1: L2norm = 0.0412412582143466
Step 2: L2norm = 0.0012786882600204618
Step 3: L2norm = 6.549574546899495e-05
Step 4: L2norm = 3.461268001484957e-06
Step 5: L2norm = 1.8835042692277217e-07
Step 6: L2norm = 1.0608943156859026e-08
Step 7: L2norm = 6.304319240618291e-10
Step 8: L2norm = 8.649396765523875e-11


 ## iFEM

In [15]:
pmesh_CaseB_iFEM, flap_mesh_CaseB_iFEM = src.hiFEM(
    verts3D=resection_verts_CaseB,
    faces=resection_faces_CaseB,
    thickness=2.0,
    nu=0.49,
    material_model="Hollman",
    max_iters=30,
    convergence_tol=1e-10,
    verbose=True,
    scene_obj=scene_obj_CaseB,
)
# Output file: result_flap_CaseB_iFEM.obj
pmesh_CaseB_iFEM.save("outputs/result_flap_CaseB_iFEM.obj", binary=False)


Step 0: L2norm = 87.42494831406165
Step 1: L2norm = 0.6821378492742431
Step 2: L2norm = 0.31239446346481736
Step 3: L2norm = 0.18060617908488732
Step 4: L2norm = 0.12877434876458532
Step 5: L2norm = 0.09465559596004346
Step 6: L2norm = 0.08050351375199805
Step 7: L2norm = 0.06877123155236438
Step 8: L2norm = 0.06454314543078245
Step 9: L2norm = 0.059558693875544175
Step 10: L2norm = 0.05888492729398661
Step 11: L2norm = 0.05695196167994276
Step 12: L2norm = 0.05833124860509369
Step 13: L2norm = 0.0585256144840307
Step 14: L2norm = 0.061424967197068824
Step 15: L2norm = 0.06336817951800465
Step 16: L2norm = 0.06747073817950312
Step 17: L2norm = 0.07090775478269601
Step 18: L2norm = 0.07602890930131115
Step 19: L2norm = 0.08079663788463178
Step 20: L2norm = 0.08687434564420592
Step 21: L2norm = 0.09290322986618259
Step 22: L2norm = 0.09997510365367696
Step 23: L2norm = 0.10727305317386528
Step 24: L2norm = 0.1154518003198987
Step 25: L2norm = 0.12408128822800757
Step 26: L2norm = 0.13353

 ## BFF

In [16]:
pmesh_CaseB_BFF, flap_mesh_CaseB_BFF = src.hiFEM(
    verts3D=resection_verts_CaseB,
    faces=resection_faces_CaseB,
    thickness=2.0,
    nu=0.49,
    fix_all_dofs=True,
    max_iters=30,
    convergence_tol=1e-10,
    verbose=True,
    scene_obj=scene_obj_CaseB,
)
# Output file: result_flap_CaseB_BFF.obj
pmesh_CaseB_BFF.save("outputs/result_flap_CaseB_BFF.obj", binary=False)


Step 0: L2norm = 87.39409768310213
Step 1: L2norm = 0.0


 ## NURBS flattening

In [17]:
# Load the specific resection surface associated with the NURBS generation process.
resection_mesh_NURBS_CaseB = pv.read(Path("inputs/CaseB_NURBS_resection.obj")).triangulate()
resection_verts_NURBS_CaseB = resection_mesh_NURBS_CaseB.points
resection_faces_NURBS_CaseB = resection_mesh_NURBS_CaseB.faces.reshape(-1, 4)[:, 1:]

# Load the NURBS-generated flattened flap vertices.
ig_verts_CaseB = pv.read(Path("inputs/CaseB_NURBS_flap.obj")).triangulate().points[:, :2]

# Only calculate deformation metrics for the provided NURBS flap
pmesh_CaseB_NURBS, flap_mesh_CaseB_NURBS = src.hiFEM(
    verts3D=resection_verts_NURBS_CaseB,
    faces=resection_faces_NURBS_CaseB,
    thickness=2.0,
    nu=0.49,
    ig_verts=ig_verts_CaseB,
    fix_all_dofs=True,
    max_iters=30,
    convergence_tol=1e-10,
    verbose=True,
    scene_obj=scene_obj_CaseB,
)
# Output file: result_flap_CaseB_NURBS.obj
pmesh_CaseB_NURBS.save("outputs/result_flap_CaseB_NURBS.obj", binary=False)



Step 0: L2norm = 128.3328603335104
Step 1: L2norm = 0.0


 # Case C

 ## Load common inputs

In [18]:
# Load the main resection surface
resection_mesh_CaseC = pv.read(Path("inputs/CaseC_Resection.obj")).triangulate()
resection_verts_CaseC = resection_mesh_CaseC.points
resection_faces_CaseC = resection_mesh_CaseC.faces.reshape(-1, 4)[:, 1:]

# Load the scene object
scene_obj_CaseC = pv.read(Path("inputs/CaseC_TongueResected.obj"))


 ## hiFEM

In [19]:
pmesh_CaseC_hiFEM, flap_mesh_CaseC_hiFEM = src.hiFEM(
    verts3D=resection_verts_CaseC,
    faces=resection_faces_CaseC,
    thickness=2.0,
    nu=0.49,
    max_iters=30,
    convergence_tol=1e-10,
    verbose=True,
    scene_obj=scene_obj_CaseC,
)
# Output file: result_flap_CaseC_hiFEM.obj
pmesh_CaseC_hiFEM.save("outputs/result_flap_CaseC_hiFEM.obj", binary=False)


Step 0: L2norm = 78.13146476472681
Step 1: L2norm = 0.051813066168395455
Step 2: L2norm = 0.0029711238826762784
Step 3: L2norm = 0.00020538216865252236
Step 4: L2norm = 1.3965670774910877e-05
Step 5: L2norm = 9.215788278696639e-07
Step 6: L2norm = 5.923178897897005e-08
Step 7: L2norm = 3.7423425590297605e-09
Step 8: L2norm = 2.40864092945487e-10
Step 9: L2norm = 2.177143342586086e-11


 ## iFEM

In [20]:
pmesh_CaseC_iFEM, flap_mesh_CaseC_iFEM = src.hiFEM(
    verts3D=resection_verts_CaseC,
    faces=resection_faces_CaseC,
    thickness=2.0,
    nu=0.49,
    material_model="Hollman",
    max_iters=30,
    convergence_tol=1e-10,
    verbose=True,
    scene_obj=scene_obj_CaseC,
)
# Output file: result_flap_CaseC_iFEM.obj
pmesh_CaseC_iFEM.save("outputs/result_flap_CaseC_iFEM.obj", binary=False)


Step 0: L2norm = 78.01165944618351
Step 1: L2norm = 0.5898863591204002
Step 2: L2norm = 0.24145158540386039
Step 3: L2norm = 0.15672516346949195
Step 4: L2norm = 0.14405821851577735
Step 5: L2norm = 0.12146325295249055
Step 6: L2norm = 0.1268260215575561
Step 7: L2norm = 0.11579611901037015
Step 8: L2norm = 0.12909265627178948
Step 9: L2norm = 0.12287658034138718
Step 10: L2norm = 0.14192609727629724
Step 11: L2norm = 0.13762763653519774
Step 12: L2norm = 0.16216378632709647
Step 13: L2norm = 0.15792220530670664
Step 14: L2norm = 0.1887070086601177
Step 15: L2norm = 0.1829058784243646
Step 16: L2norm = 0.22136006598327748
Step 17: L2norm = 0.21226437864912787
Step 18: L2norm = 0.2602757242766763
Step 19: L2norm = 0.24585156266555955
Step 20: L2norm = 0.30562084332415657
Step 21: L2norm = 0.2834841585080099
Step 22: L2norm = 0.3573197552099955
Step 23: L2norm = 0.3248186432307357
Step 24: L2norm = 0.414839717031916
Step 25: L2norm = 0.3692812379310386
Step 26: L2norm = 0.477053167404605

 ## BFF

In [21]:
pmesh_CaseC_BFF, flap_mesh_CaseC_BFF = src.hiFEM(
    verts3D=resection_verts_CaseC,
    faces=resection_faces_CaseC,
    thickness=2.0,
    nu=0.49,
    fix_all_dofs=True,
    max_iters=30,
    convergence_tol=1e-10,
    verbose=True,
    scene_obj=scene_obj_CaseC,
)
# Output file: result_flap_CaseC_BFF.obj
pmesh_CaseC_BFF.save("outputs/result_flap_CaseC_BFF.obj", binary=False)


Step 0: L2norm = 78.07024358862876
Step 1: L2norm = 0.0


 ## NURBS flattening (Metrics Calculation)

In [22]:
# Load the specific resection surface associated with the NURBS generation process.
resection_mesh_NURBS_CaseC = pv.read(Path("inputs/CaseC_NURBS_resection.obj")).triangulate()
resection_verts_NURBS_CaseC = resection_mesh_NURBS_CaseC.points
resection_faces_NURBS_CaseC = resection_mesh_NURBS_CaseC.faces.reshape(-1, 4)[:, 1:]

# Load the NURBS-generated flattened flap vertices.
ig_verts_CaseC = pv.read(Path("inputs/CaseC_NURBS_flap.obj")).triangulate().points[:, :2]

# Only calculate deformation metrics for the provided NURBS flap
pmesh_CaseC_NURBS, flap_mesh_CaseC_NURBS = src.hiFEM(
    verts3D=resection_verts_NURBS_CaseC,
    faces=resection_faces_NURBS_CaseC,
    thickness=2.0,
    nu=0.49,
    ig_verts=ig_verts_CaseC,
    fix_all_dofs=True,
    max_iters=30,
    convergence_tol=1e-10,
    verbose=True,
    scene_obj=scene_obj_CaseC,
)
# Output file: result_flap_CaseC_NURBS.obj
pmesh_CaseC_NURBS.save("outputs/result_flap_CaseC_NURBS.obj", binary=False)



Step 0: L2norm = 129.40844999019964
Step 1: L2norm = 0.0


 # Case D

 ## Load common inputs

In [23]:
# Load the main resection surface
resection_mesh_CaseD = pv.read(Path("inputs/CaseD_Resection.obj")).triangulate()
resection_verts_CaseD = resection_mesh_CaseD.points
resection_faces_CaseD = resection_mesh_CaseD.faces.reshape(-1, 4)[:, 1:]

# Load the scene object
scene_obj_CaseD = pv.read(Path("inputs/CaseD_TongueResected.obj"))


 ## hiFEM

In [24]:
pmesh_CaseD_hiFEM, flap_mesh_CaseD_hiFEM = src.hiFEM(
    verts3D=resection_verts_CaseD,
    faces=resection_faces_CaseD,
    thickness=2.0,
    nu=0.49,
    max_iters=30,
    convergence_tol=1e-10,
    verbose=True,
    scene_obj=scene_obj_CaseD,
)
# Output file: result_flap_CaseD_hiFEM.obj
pmesh_CaseD_hiFEM.save("outputs/result_flap_CaseD_hiFEM.obj", binary=False)


Step 0: L2norm = 61.125290052036405
Step 1: L2norm = 0.031008387033500277
Step 2: L2norm = 0.001202579540001278
Step 3: L2norm = 6.247275950122478e-05
Step 4: L2norm = 3.5765988554573673e-06
Step 5: L2norm = 2.1589379449220135e-07
Step 6: L2norm = 1.3893188145532428e-08
Step 7: L2norm = 9.214892193331897e-10
Step 8: L2norm = 7.047202609009589e-11


 ## BFF

In [25]:
pmesh_CaseD_BFF, flap_mesh_CaseD_BFF = src.hiFEM(
    verts3D=resection_verts_CaseD,
    faces=resection_faces_CaseD,
    thickness=2.0,
    nu=0.49,
    fix_all_dofs=True,
    max_iters=30,
    convergence_tol=1e-10,
    verbose=True,
    scene_obj=scene_obj_CaseD,
)
# Output file: result_flap_CaseD_BFF.obj
pmesh_CaseD_BFF.save("outputs/result_flap_CaseD_BFF.obj", binary=False)


Step 0: L2norm = 61.0895679634649
Step 1: L2norm = 0.0


 ## NURBS flattening (Metrics Calculation)

In [26]:
# Load the specific resection surface associated with the NURBS generation process.
resection_mesh_NURBS_CaseD = pv.read(Path("inputs/CaseD_NURBS_resection.obj")).triangulate()
resection_verts_NURBS_CaseD = resection_mesh_NURBS_CaseD.points
resection_faces_NURBS_CaseD = resection_mesh_NURBS_CaseD.faces.reshape(-1, 4)[:, 1:]

# Load the NURBS-generated flattened flap vertices.
ig_verts_CaseD = pv.read(Path("inputs/CaseD_NURBS_flap.obj")).triangulate().points[:, :2]

# Only calculate deformation metrics for the provided NURBS flap
pmesh_CaseD_NURBS, flap_mesh_CaseD_NURBS = src.hiFEM(
    verts3D=resection_verts_NURBS_CaseD,
    faces=resection_faces_NURBS_CaseD,
    thickness=2.0,
    nu=0.49,
    ig_verts=ig_verts_CaseD,
    fix_all_dofs=True,
    max_iters=30,
    convergence_tol=1e-10,
    verbose=True,
    scene_obj=scene_obj_CaseD,
)
# Output file: result_flap_CaseD_NURBS.obj
pmesh_CaseD_NURBS.save("outputs/result_flap_CaseD_NURBS.obj", binary=False)


Step 0: L2norm = 102.91720865073293
Step 1: L2norm = 0.0
